In [4]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [5]:
!ls "/gdrive/My Drive"

 데이터사이언스메뉴얼		      '코로나 데이터 셋 설명.gdoc'
'0405 학술제 팀별 진행계획.gdoc'      '데이콘 코로나 시각화 공모전.gdoc'
 코로나2주차주제_0407.gdoc	      '학술제 타임테이블.gdoc'
'bhc해바라기 봉사단 4월 계획서.gdoc'  '데이터사이언스 메뉴얼.gdoc'
 cifar-class			       mnist
'Colab Notebooks'		       큐시즘_경쟁pt
 denoising-dirty-documents	       python_study.ipynb
'제목 없는 문서.gdoc'		       Untitled0.ipynb


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import torch
import torchvision
import glob
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
 
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image

In [0]:
TRAIN_IMAGES = glob.glob('/gdrive/My Drive/denoising-dirty-documents/train/train/*.png')
TRAIN_CLEAN = glob.glob('/gdrive/My Drive/denoising-dirty-documents/train_cleaned/train_cleaned/*.png')
TEST_IMAGES = glob.glob('/gdrive/My Drive/denoising-dirty-documents/test/test/*.png')

In [0]:
def get_data(path):
    images = []
    for i, image in enumerate(path):
        image = cv2.imread(image)
        image = cv2.resize(image, (256, 256))
        image = np.reshape(image, image.shape[0]*image.shape[1]*image.shape[2])
        images.append(image)
    return images
 
x_train = get_data(TRAIN_IMAGES)
y_train = get_data(TRAIN_CLEAN)
x_test = get_data(TEST_IMAGES)

In [9]:

print(len(x_train))
print(len(y_train))
print(len(x_test))
 
print(x_train[1])

144
144
72
[225 225 225 ... 215 215 215]


In [0]:
# constants
NUM_EPOCHS = 100
LEARNING_RATE = 1e-3
BATCH_SIZE = 2

In [0]:

# define transforms
transform = transforms.Compose([
     transforms.ToPILImage(),
     transforms.ToTensor()
])

In [0]:
# prepare the dataset and the dataloader
class ImageData(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i][:]
        data = np.asarray(data).astype(np.uint8).reshape((256, 256, 3))
        
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            labels = self.y[i][:]
            labels = np.asarray(labels).astype(np.uint8).reshape((256, 256, 3))
            labels = self.transforms(labels)
            return (data, labels)
        else:
            return data
        
train_data = ImageData(x_train, y_train, transform)
test_data = ImageData(x_test, None, transform)
 
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)